In [1]:
from transformers import  BertModel, BertConfig,BertTokenizer
from transformers import BertTokenizer,DataCollatorWithPadding, TrainingArguments
from transformers import BertForTokenClassification,BertModel
from transformers import Trainer
from datasets import Dataset, DatasetDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
single_modal_inputs = {
"source_sentence": ['杭州余杭东方未来学校附近世纪华联商场(金家渡北苑店)'],
"sentences_to_compare": [[
'良渚街道金家渡北苑42号世纪华联超市(金家渡北苑店)',
'金家渡路金家渡中苑南区70幢金家渡中苑70幢',
'金家渡路140-142号附近家家福足道(金家渡店)'
]]
}
train_datasets = Dataset.from_dict(single_modal_inputs)
train_datasets

Dataset({
    features: ['source_sentence', 'sentences_to_compare'],
    num_rows: 1
})

In [4]:
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [5]:
train_encodings = tokenizer([example["source_sentence"] for example in iter(train_datasets)], truncation=True, padding=True)
train_encodings.keys()
#train_encodings['label'] = train_datasets['label']


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [6]:
class TextNet(nn.Module):
    def __init__(self, code_len):
        super(TextNet,self).__init__()
        #modelconfig = BertConfig.from_pretrained(model_name)
        self.textextract = BertModel.from_pretrained(model_name)
        embedding_dim = self.textextract.config.hidden_size
        print(embedding_dim)#768
        print(self.textextract.config)
        self.fc = nn.Linear(embedding_dim, code_len)
        self.tanh = nn.Tanh()
    def forward(self,tokens, segments, input_masks):
        out = self.textextract(tokens, token_type_ids=segments,
                                 		attention_mask=input_masks)
        text_embeddings = out[0][:,0,:]#output[0](batch size, sequence length, model hidden dimension)这里表示取cls的embedding
        features = self.fc(text_embeddings)
        features=self.tanh(features)
        return text_embeddings

model = TextNet(256)


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768
BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}



In [39]:
embedding1 = model(torch.tensor(train_encodings['input_ids']),torch.tensor( train_encodings['token_type_ids']),
                      torch.tensor(train_encodings['attention_mask']))

In [40]:
test_encodings = tokenizer([example["sentences_to_compare"][0] for example in iter(train_datasets)], truncation=True, padding=True)
test_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [41]:
embedding2 = model(torch.tensor(test_encodings['input_ids']),torch.tensor( test_encodings['token_type_ids']),
                      torch.tensor(test_encodings['attention_mask']))

In [42]:
test_encodings = tokenizer([example["sentences_to_compare"][1] for example in iter(train_datasets)], truncation=True, padding=True)
test_encodings.keys()
embedding3 = model(torch.tensor(test_encodings['input_ids']),torch.tensor( test_encodings['token_type_ids']),
                      torch.tensor(test_encodings['attention_mask']))

In [45]:
def get_cos_similar(v1: list, v2: list):
    num = float(np.dot(v1, v2))  # 向量点乘
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)  # 求模长的乘积
    return 0.5 + 0.5 * (num / denom) if denom != 0 else 0
get_cos_similar(list(embedding3.detach().numpy()[0]),list(embedding1.detach().numpy()[0]))

0.9055967626981778